In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

### Read in the required datasets

In [2]:
#### read the required data files

## counts
fnl_large_core_dm_counts_90_02 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_90_02/final_main_large_cities_core_counts_dm_1990_2002.csv')
fnl_medium_core_dm_counts_90_02 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_90_02/final_main_medium_cities_core_counts_dm_1990_2002.csv')
fnl_small_core_dm_counts_90_02 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_90_02/final_main_small_cities_core_counts_dm_1990_2002.csv')

fnl_large_core_dm_counts_03_15 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_03_15/final_main_large_cities_core_counts_dm_2003_2015.csv')
fnl_medium_core_dm_counts_03_15 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_03_15/final_main_medium_cities_core_counts_dm_2003_2015.csv')
fnl_small_core_dm_counts_03_15 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_03_15/final_main_small_cities_core_counts_dm_2003_2015.csv')

## rates
fnl_large_core_dm_rates_90_02 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_90_02/final_main_large_cities_core_rates_dm_1990_2002.csv')
fnl_medium_core_dm_rates_90_02 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_90_02/final_main_medium_cities_core_rates_dm_1990_2002.csv')
fnl_small_core_dm_rates_90_02 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_90_02/final_main_small_cities_core_rates_dm_1990_2002.csv')

fnl_large_core_dm_rates_03_15 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_03_15/final_main_large_cities_core_rates_dm_2003_2015.csv')
fnl_medium_core_dm_rates_03_15 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_03_15/final_main_medium_cities_core_rates_dm_2003_2015.csv')
fnl_small_core_dm_rates_03_15 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_03_15/final_main_small_cities_core_rates_dm_2003_2015.csv')

#### get median pop to be used as weights

In [3]:
def create_pop_wt_col_by_median(req_dfs):
    for df in req_dfs:
        # returns a df pop_med
        df_req = df.loc[:, ['ORI', 'fnl_population']]
        df_req_med = df_req.groupby('ORI').agg({'fnl_population':np.median})
        df_req_med.rename({'fnl_population': 'fnl_population_wt'}, axis=1, inplace=True)
        df = df.merge(df_req_med, on='ORI', how='left')
        return df

In [4]:
#### Create pop wt col for dm counts
fnl_large_core_dm_counts_90_02 = create_pop_wt_col_by_median(req_dfs=[fnl_large_core_dm_counts_90_02])
fnl_medium_core_dm_counts_90_02 = create_pop_wt_col_by_median(req_dfs=[fnl_medium_core_dm_counts_90_02])
fnl_small_core_dm_counts_90_02 = create_pop_wt_col_by_median(req_dfs=[fnl_small_core_dm_counts_90_02])

fnl_large_core_dm_counts_03_15 = create_pop_wt_col_by_median(req_dfs=[fnl_large_core_dm_counts_03_15])
fnl_medium_core_dm_counts_03_15 = create_pop_wt_col_by_median(req_dfs=[fnl_medium_core_dm_counts_03_15])
fnl_small_core_dm_counts_03_15 = create_pop_wt_col_by_median(req_dfs=[fnl_small_core_dm_counts_03_15])

In [5]:
#### Create pop wt col for dm rates
fnl_large_core_dm_rates_90_02 = create_pop_wt_col_by_median(req_dfs=[fnl_large_core_dm_rates_90_02])
fnl_medium_core_dm_rates_90_02 = create_pop_wt_col_by_median(req_dfs=[fnl_medium_core_dm_rates_90_02])
fnl_small_core_dm_rates_90_02 = create_pop_wt_col_by_median(req_dfs=[fnl_small_core_dm_rates_90_02])

fnl_large_core_dm_rates_03_15 = create_pop_wt_col_by_median(req_dfs=[fnl_large_core_dm_rates_03_15])
fnl_medium_core_dm_rates_03_15 = create_pop_wt_col_by_median(req_dfs=[fnl_medium_core_dm_rates_03_15])
fnl_small_core_dm_rates_03_15 = create_pop_wt_col_by_median(req_dfs=[fnl_small_core_dm_rates_03_15])

#### func to create lagged vars with reqd shift

In [6]:
def create_lag_vars(req_dfs, vars_to_lag, groupby_var, num_of_lags):
    for df in req_dfs:
        count=1
        for lag_num in range(num_of_lags):
            for var in vars_to_lag:
                df[f'lag{count}_{var}'] = df.groupby(f'{groupby_var}')[f'{var}'].shift(count)
            count+=1
        return df

In [7]:
#### create lags for counts
vars_lag_list_counts = ['dm_violent_crime', 'dm_total_officers', 'dm_prison_occupancy_count', 'dm_jail_occupancy_count', 'dm_drug_tot_arrests', 
               'dm_disorder_arrests_tot_index']

fnl_large_core_dm_counts_90_02 = create_lag_vars(req_dfs=[fnl_large_core_dm_counts_90_02],
                         vars_to_lag=vars_lag_list_counts,
                         groupby_var = 'ORI',
                         num_of_lags=2)
                                                     
fnl_medium_core_dm_counts_90_02 = create_lag_vars(req_dfs=[fnl_medium_core_dm_counts_90_02],
                         vars_to_lag=vars_lag_list_counts,
                         groupby_var = 'ORI',
                         num_of_lags=2)
                                                          
fnl_small_core_dm_counts_90_02 = create_lag_vars(req_dfs=[fnl_small_core_dm_counts_90_02],
                         vars_to_lag=vars_lag_list_counts,
                         groupby_var = 'ORI',
                         num_of_lags=2)
                
fnl_large_core_dm_counts_03_15 = create_lag_vars(req_dfs=[fnl_large_core_dm_counts_03_15],
                         vars_to_lag=vars_lag_list_counts,
                         groupby_var = 'ORI',
                         num_of_lags=2)
                                                     
fnl_medium_core_dm_counts_03_15 = create_lag_vars(req_dfs=[fnl_medium_core_dm_counts_03_15],
                         vars_to_lag=vars_lag_list_counts,
                         groupby_var = 'ORI',
                         num_of_lags=2)
                                                          
fnl_small_core_dm_counts_03_15 = create_lag_vars(req_dfs=[fnl_small_core_dm_counts_03_15],
                         vars_to_lag=vars_lag_list_counts,
                         groupby_var = 'ORI',
                         num_of_lags=2)

In [29]:
#### create lags for rates
vars_lag_list_rates = ['dm_violent_crime_rate', 'dm_total_officers_rate', 'dm_prison_occupancy_count_rate', 'dm_jail_occupancy_count_rate',
               'dm_drug_tot_arrests_rate', 'dm_disorder_arrests_tot_index_rate']

fnl_large_core_dm_rates_90_02 = create_lag_vars(req_dfs=[fnl_large_core_dm_rates_90_02],
                         vars_to_lag=vars_lag_list_rates,
                         groupby_var = 'ORI',
                         num_of_lags=2)


fnl_large_core_dm_rates_90_02.to_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_pop_ref_lag/fnl_large_core_dm_rates_lag_90_02.csv',
                                    index=False)

                                                     
fnl_medium_core_dm_rates_90_02 = create_lag_vars(req_dfs=[fnl_medium_core_dm_rates_90_02],
                         vars_to_lag=vars_lag_list_rates,
                         groupby_var = 'ORI',
                         num_of_lags=2)

fnl_medium_core_dm_rates_90_02.to_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_pop_ref_lag/fnl_medium_core_dm_rates_lag_90_02.csv',
                                    index=False)

                                                          
fnl_small_core_dm_rates_90_02 = create_lag_vars(req_dfs=[fnl_small_core_dm_rates_90_02],
                         vars_to_lag=vars_lag_list_rates,
                         groupby_var = 'ORI',
                         num_of_lags=2)

fnl_small_core_dm_rates_90_02.to_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_pop_ref_lag/fnl_small_core_dm_rates_lag_90_02.csv',
                                    index=False)


                
fnl_large_core_dm_rates_03_15 = create_lag_vars(req_dfs=[fnl_large_core_dm_rates_03_15],
                         vars_to_lag=vars_lag_list_rates,
                         groupby_var = 'ORI',
                         num_of_lags=2)

fnl_large_core_dm_rates_03_15.to_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_pop_ref_lag/fnl_large_core_dm_rates_lag_03_15.csv',
                                    index=False)

                                                     
fnl_medium_core_dm_rates_03_15 = create_lag_vars(req_dfs=[fnl_medium_core_dm_rates_03_15],
                         vars_to_lag=vars_lag_list_rates,
                         groupby_var = 'ORI',
                         num_of_lags=2)

fnl_medium_core_dm_rates_03_15.to_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_pop_ref_lag/fnl_medium_core_dm_rates_lag_03_15.csv',
                                    index=False)

                                                          
fnl_small_core_dm_rates_03_15 = create_lag_vars(req_dfs=[fnl_small_core_dm_rates_03_15],
                         vars_to_lag=vars_lag_list_rates,
                         groupby_var = 'ORI',
                         num_of_lags=2)

fnl_small_core_dm_rates_03_15.to_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_pop_ref_lag/fnl_small_core_dm_rates_lag_03_15.csv',
                                    index=False)

#### get agency level 90-15 files

In [17]:
fnl_large_core_dm_counts_lag_90_15 = fnl_large_core_dm_counts_90_02.append([fnl_large_core_dm_counts_03_15], sort=False)

fnl_large_core_dm_counts_lag_90_15.sort_values(['ORI', 'YEAR'], inplace=True)

fnl_large_core_dm_counts_lag_90_15.to_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_pop_ref_lag/fnl_large_core_dm_counts_90_15.csv',
                               index=False)


fnl_medium_core_dm_counts_lag_90_15 = fnl_medium_core_dm_counts_90_02.append([fnl_medium_core_dm_counts_03_15], sort=False)

fnl_medium_core_dm_counts_lag_90_15.sort_values(['ORI', 'YEAR'], inplace=True)

fnl_medium_core_dm_counts_lag_90_15.to_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_pop_ref_lag/fnl_medium_core_dm_counts_90_15.csv',
                               index=False)


fnl_small_core_dm_counts_lag_90_15 = fnl_small_core_dm_counts_90_02.append([fnl_small_core_dm_counts_03_15], sort=False)

fnl_small_core_dm_counts_lag_90_15.sort_values(['ORI', 'YEAR'], inplace=True)

fnl_small_core_dm_counts_lag_90_15.to_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_pop_ref_lag/fnl_small_core_dm_counts_90_15.csv',
                               index=False)




fnl_large_core_dm_rates_lag_90_15 = fnl_large_core_dm_rates_90_02.append([fnl_large_core_dm_rates_03_15], sort=False)

fnl_large_core_dm_rates_lag_90_15.sort_values(['ORI', 'YEAR'], inplace=True)

fnl_large_core_dm_rates_lag_90_15.to_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_pop_ref_lag/fnl_large_core_dm_rates_90_15.csv',
                               index=False)


fnl_medium_core_dm_rates_lag_90_15 = fnl_medium_core_dm_rates_90_02.append([fnl_medium_core_dm_rates_03_15], sort=False)

fnl_medium_core_dm_rates_lag_90_15.sort_values(['ORI', 'YEAR'], inplace=True)

fnl_medium_core_dm_rates_lag_90_15.to_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_pop_ref_lag/fnl_medium_core_dm_rates_90_15.csv',
                               index=False)


fnl_small_core_dm_rates_lag_90_15 = fnl_small_core_dm_rates_90_02.append([fnl_small_core_dm_rates_03_15], sort=False)

fnl_small_core_dm_rates_lag_90_15.sort_values(['ORI', 'YEAR'], inplace=True)

fnl_small_core_dm_rates_lag_90_15.to_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_pop_ref_lag/fnl_small_core_dm_rates_90_15.csv',
                               index=False)

#### append all individual counts files to get top level final files

In [9]:
fnl_core_counts_dm_pop_ref_wt_lag_90_02 = fnl_large_core_dm_counts_90_02.append([fnl_medium_core_dm_counts_90_02, fnl_small_core_dm_counts_90_02], sort=False)

fnl_core_counts_dm_pop_ref_wt_lag_90_02.sort_values(['ORI', 'YEAR'], inplace=True)

fnl_core_counts_dm_pop_ref_wt_lag_90_02.to_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_pop_ref_lag/fnl_core_counts_dm_pop_ref_wt_lag_90_02.csv',
                               index=False)


fnl_core_counts_dm_pop_ref_wt_lag_03_15 = fnl_large_core_dm_counts_03_15.append([fnl_medium_core_dm_counts_03_15, fnl_small_core_dm_counts_03_15], sort=False)

fnl_core_counts_dm_pop_ref_wt_lag_03_15.sort_values(['ORI', 'YEAR'], inplace=True)

fnl_core_counts_dm_pop_ref_wt_lag_03_15.to_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_pop_ref_lag/fnl_core_counts_dm_pop_ref_wt_lag_03_15.csv',
                               index=False)


fnl_core_counts_dm_pop_ref_wt_lag_90_15 = fnl_core_counts_dm_pop_ref_wt_lag_90_02.append([fnl_core_counts_dm_pop_ref_wt_lag_03_15], sort=False)

fnl_core_counts_dm_pop_ref_wt_lag_90_15.sort_values(['ORI', 'YEAR'], inplace=True)

fnl_core_counts_dm_pop_ref_wt_lag_90_15.to_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_pop_ref_lag/fnl_core_counts_dm_pop_ref_wt_lag_90_15.csv',
                               index=False)

#### append all individual rates files to get top level final files


In [10]:
fnl_core_rates_dm_pop_ref_wt_lag_90_02 = fnl_large_core_dm_rates_90_02.append([fnl_medium_core_dm_rates_90_02, fnl_small_core_dm_rates_90_02], sort=False)

fnl_core_rates_dm_pop_ref_wt_lag_90_02.sort_values(['ORI', 'YEAR'], inplace=True)

fnl_core_rates_dm_pop_ref_wt_lag_90_02.to_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_pop_ref_lag/fnl_core_rates_dm_pop_ref_wt_lag_90_02.csv',
                               index=False)


fnl_core_rates_dm_pop_ref_wt_lag_03_15 = fnl_large_core_dm_rates_03_15.append([fnl_medium_core_dm_rates_03_15, fnl_small_core_dm_rates_03_15], sort=False)

fnl_core_rates_dm_pop_ref_wt_lag_03_15.sort_values(['ORI', 'YEAR'], inplace=True)

fnl_core_rates_dm_pop_ref_wt_lag_03_15.to_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_pop_ref_lag/fnl_core_rates_dm_pop_ref_wt_lag_03_15.csv',
                               index=False)


fnl_core_rates_dm_pop_ref_wt_lag_90_15 = fnl_core_rates_dm_pop_ref_wt_lag_90_02.append([fnl_core_rates_dm_pop_ref_wt_lag_03_15], sort=False)

fnl_core_rates_dm_pop_ref_wt_lag_90_15.sort_values(['ORI', 'YEAR'], inplace=True)

fnl_core_rates_dm_pop_ref_wt_lag_90_15.to_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_pop_ref_lag/fnl_core_rates_dm_pop_ref_wt_lag_90_15.csv',
                               index=False)

#### create a func to automate weighted lagged regression - univariate for now

In [11]:
def wols_lagged_uni(req_dfs, dep_var, ind_vars, weights_var, dataset_names=False):
    for df in req_dfs:
        for var in ind_vars:
            model = sm.WLS.from_formula(f'{dep_var} ~ {var} + lag1_{var} + lag2_{var}', data=df, weights=df[f'{weights_var}'])
            results = model.fit()
            if dataset_names:
                print(dataset_names[df_count])
            print(results.summary(), '\n', '\n')


In [12]:
dm_dep_var_rates = 'dm_violent_crime_rate'
dm_ind_vars_rates = ['dm_total_officers_rate', 'dm_prison_occupancy_count_rate', 'dm_jail_occupancy_count_rate',
               'dm_drug_tot_arrests_rate', 'dm_disorder_arrests_tot_index_rate']

#### Rates - Perform wls with ref pop wts and lag for fnl_core_rates_dm_pop_ref_wt_lag_90_15.csv file

In [13]:
wols_lagged_uni(req_dfs=[fnl_core_rates_dm_pop_ref_wt_lag_90_15], 
                dep_var=dm_dep_var_rates,
               ind_vars=dm_ind_vars_rates,
               weights_var='fnl_population_wt')

                              WLS Regression Results                             
Dep. Variable:     dm_violent_crime_rate   R-squared:                       0.008
Model:                               WLS   Adj. R-squared:                  0.008
Method:                    Least Squares   F-statistic:                     230.8
Date:                   Sat, 04 Apr 2020   Prob (F-statistic):          3.70e-149
Time:                           21:28:49   Log-Likelihood:            -6.5339e+05
No. Observations:                  91031   AIC:                         1.307e+06
Df Residuals:                      91027   BIC:                         1.307e+06
Df Model:                              3                                         
Covariance Type:               nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------

#### Rates - Perform wls with ref pop wts and lag for fnl_core_rates_dm_pop_ref_wt_lag_90_02.csv file

In [14]:
wols_lagged_uni(req_dfs=[fnl_core_rates_dm_pop_ref_wt_lag_90_02], 
                dep_var=dm_dep_var_rates,
               ind_vars=dm_ind_vars_rates,
               weights_var='fnl_population_wt')

                              WLS Regression Results                             
Dep. Variable:     dm_violent_crime_rate   R-squared:                       0.012
Model:                               WLS   Adj. R-squared:                  0.012
Method:                    Least Squares   F-statistic:                     178.8
Date:                   Sat, 04 Apr 2020   Prob (F-statistic):          3.44e-115
Time:                           21:28:49   Log-Likelihood:            -3.0901e+05
No. Observations:                  42659   AIC:                         6.180e+05
Df Residuals:                      42655   BIC:                         6.181e+05
Df Model:                              3                                         
Covariance Type:               nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------

#### Rates - Perform wls with ref pop wts and lag for fnl_core_rates_dm_pop_ref_wt_lag_03_15.csv file

In [15]:
wols_lagged_uni(req_dfs=[fnl_core_rates_dm_pop_ref_wt_lag_03_15], 
                dep_var=dm_dep_var_rates,
               ind_vars=dm_ind_vars_rates,
               weights_var='fnl_population_wt')

                              WLS Regression Results                             
Dep. Variable:     dm_violent_crime_rate   R-squared:                       0.002
Model:                               WLS   Adj. R-squared:                  0.002
Method:                    Least Squares   F-statistic:                     28.63
Date:                   Sat, 04 Apr 2020   Prob (F-statistic):           1.74e-18
Time:                           21:36:19   Log-Likelihood:            -3.3733e+05
No. Observations:                  48372   AIC:                         6.747e+05
Df Residuals:                      48368   BIC:                         6.747e+05
Df Model:                              3                                         
Covariance Type:               nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------

#### Rates - Large - 90 -15

In [18]:
wols_lagged_uni(req_dfs=[fnl_large_core_dm_rates_lag_90_15], 
                dep_var=dm_dep_var_rates,
               ind_vars=dm_ind_vars_rates,
               weights_var='fnl_population_wt')

                              WLS Regression Results                             
Dep. Variable:     dm_violent_crime_rate   R-squared:                       0.041
Model:                               WLS   Adj. R-squared:                  0.041
Method:                    Least Squares   F-statistic:                     109.3
Date:                   Sat, 04 Apr 2020   Prob (F-statistic):           2.88e-69
Time:                           22:22:54   Log-Likelihood:                -55272.
No. Observations:                   7629   AIC:                         1.106e+05
Df Residuals:                       7625   BIC:                         1.106e+05
Df Model:                              3                                         
Covariance Type:               nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------

#### Rates - Medium - 90 -15

In [19]:
wols_lagged_uni(req_dfs=[fnl_medium_core_dm_rates_lag_90_15], 
                dep_var=dm_dep_var_rates,
               ind_vars=dm_ind_vars_rates,
               weights_var='fnl_population_wt')

                              WLS Regression Results                             
Dep. Variable:     dm_violent_crime_rate   R-squared:                       0.001
Model:                               WLS   Adj. R-squared:                  0.001
Method:                    Least Squares   F-statistic:                     5.604
Date:                   Sat, 04 Apr 2020   Prob (F-statistic):           0.000776
Time:                           22:26:25   Log-Likelihood:                -83704.
No. Observations:                  12630   AIC:                         1.674e+05
Df Residuals:                      12626   BIC:                         1.674e+05
Df Model:                              3                                         
Covariance Type:               nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------

#### Rates - Small - 90 -15

In [20]:
wols_lagged_uni(req_dfs=[fnl_small_core_dm_rates_lag_90_15], 
                dep_var=dm_dep_var_rates,
               ind_vars=dm_ind_vars_rates,
               weights_var='fnl_population_wt')

                              WLS Regression Results                             
Dep. Variable:     dm_violent_crime_rate   R-squared:                       0.002
Model:                               WLS   Adj. R-squared:                  0.002
Method:                    Least Squares   F-statistic:                     55.04
Date:                   Sat, 04 Apr 2020   Prob (F-statistic):           1.58e-35
Time:                           22:30:09   Log-Likelihood:            -4.6982e+05
No. Observations:                  70772   AIC:                         9.396e+05
Df Residuals:                      70768   BIC:                         9.397e+05
Df Model:                              3                                         
Covariance Type:               nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------

#### Rates - Large - 90 - 02

In [22]:
wols_lagged_uni(req_dfs=[fnl_large_core_dm_rates_90_02], 
                dep_var=dm_dep_var_rates,
               ind_vars=dm_ind_vars_rates,
               weights_var='fnl_population_wt')

                              WLS Regression Results                             
Dep. Variable:     dm_violent_crime_rate   R-squared:                       0.067
Model:                               WLS   Adj. R-squared:                  0.066
Method:                    Least Squares   F-statistic:                     91.35
Date:                   Sat, 04 Apr 2020   Prob (F-statistic):           4.27e-57
Time:                           22:42:49   Log-Likelihood:                -27831.
No. Observations:                   3836   AIC:                         5.567e+04
Df Residuals:                       3832   BIC:                         5.569e+04
Df Model:                              3                                         
Covariance Type:               nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------

#### Rates - Medium - 90 - 02

In [23]:
wols_lagged_uni(req_dfs=[fnl_medium_core_dm_rates_90_02], 
                dep_var=dm_dep_var_rates,
               ind_vars=dm_ind_vars_rates,
               weights_var='fnl_population_wt')

                              WLS Regression Results                             
Dep. Variable:     dm_violent_crime_rate   R-squared:                       0.000
Model:                               WLS   Adj. R-squared:                 -0.000
Method:                    Least Squares   F-statistic:                    0.6348
Date:                   Sat, 04 Apr 2020   Prob (F-statistic):              0.593
Time:                           22:47:06   Log-Likelihood:                -40467.
No. Observations:                   6039   AIC:                         8.094e+04
Df Residuals:                       6035   BIC:                         8.097e+04
Df Model:                              3                                         
Covariance Type:               nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------

#### Rates - Small - 90 - 02

In [24]:
wols_lagged_uni(req_dfs=[fnl_small_core_dm_rates_90_02], 
                dep_var=dm_dep_var_rates,
               ind_vars=dm_ind_vars_rates,
               weights_var='fnl_population_wt')

                              WLS Regression Results                             
Dep. Variable:     dm_violent_crime_rate   R-squared:                       0.000
Model:                               WLS   Adj. R-squared:                  0.000
Method:                    Least Squares   F-statistic:                     4.774
Date:                   Sat, 04 Apr 2020   Prob (F-statistic):            0.00250
Time:                           23:03:27   Log-Likelihood:            -2.2138e+05
No. Observations:                  32784   AIC:                         4.428e+05
Df Residuals:                      32780   BIC:                         4.428e+05
Df Model:                              3                                         
Covariance Type:               nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------

#### Rates - Large - 03-15

In [25]:
wols_lagged_uni(req_dfs=[fnl_large_core_dm_rates_03_15], 
                dep_var=dm_dep_var_rates,
               ind_vars=dm_ind_vars_rates,
               weights_var='fnl_population_wt')

                              WLS Regression Results                             
Dep. Variable:     dm_violent_crime_rate   R-squared:                       0.003
Model:                               WLS   Adj. R-squared:                  0.002
Method:                    Least Squares   F-statistic:                     3.487
Date:                   Sat, 04 Apr 2020   Prob (F-statistic):             0.0151
Time:                           23:06:18   Log-Likelihood:                -26521.
No. Observations:                   3793   AIC:                         5.305e+04
Df Residuals:                       3789   BIC:                         5.308e+04
Df Model:                              3                                         
Covariance Type:               nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------

#### Rates - Medium - 03-15

In [26]:
wols_lagged_uni(req_dfs=[fnl_medium_core_dm_rates_03_15], 
                dep_var=dm_dep_var_rates,
               ind_vars=dm_ind_vars_rates,
               weights_var='fnl_population_wt')

                              WLS Regression Results                             
Dep. Variable:     dm_violent_crime_rate   R-squared:                       0.004
Model:                               WLS   Adj. R-squared:                  0.004
Method:                    Least Squares   F-statistic:                     9.443
Date:                   Sat, 04 Apr 2020   Prob (F-statistic):           3.19e-06
Time:                           23:08:47   Log-Likelihood:                -42511.
No. Observations:                   6591   AIC:                         8.503e+04
Df Residuals:                       6587   BIC:                         8.506e+04
Df Model:                              3                                         
Covariance Type:               nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------

#### Rates - Small - 03-15

In [27]:
wols_lagged_uni(req_dfs=[fnl_small_core_dm_rates_03_15], 
                dep_var=dm_dep_var_rates,
               ind_vars=dm_ind_vars_rates,
               weights_var='fnl_population_wt')

                              WLS Regression Results                             
Dep. Variable:     dm_violent_crime_rate   R-squared:                       0.010
Model:                               WLS   Adj. R-squared:                  0.010
Method:                    Least Squares   F-statistic:                     124.9
Date:                   Sat, 04 Apr 2020   Prob (F-statistic):           1.70e-80
Time:                           23:08:53   Log-Likelihood:            -2.4621e+05
No. Observations:                  37988   AIC:                         4.924e+05
Df Residuals:                      37984   BIC:                         4.925e+05
Df Model:                              3                                         
Covariance Type:               nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------